In [24]:
# fashion mnist

import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader

In [ ]:
root='./fashion-mnist'
datasets.FashionMNIST(root, train=True, transform=None, download=True)
datasets.FashionMNIST(root, train=False, transform=None, download=True)

In [26]:
data_transform = transforms.ToTensor()

# 学習データを読み込む DataLoader を作成する。
train_dataset = datasets.FashionMNIST(
    root, train=True, transform=data_transform, download=True
)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True
)

# テストデータを読み込む DataLoader を作成する。
test_dataset = datasets.FashionMNIST(
    root, train=False, transform=data_transform, download=True
)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True
)

In [ ]:
import pprint

class_ids, sample_indices = np.unique(train_dataset.targets, return_index=True)
class_ids, sample_indices

In [ ]:
fig = plt.figure(figsize=(10, 4))
for i in range(10):
    ax = fig.add_subplot(2, 5, i + 1)
for i in range(10):
    print(i, train_dataset.targets[i])
    img = train_dataset.data[sample_indices[i]]
    plt.imshow(img, cmap="gray")

In [ ]:
# 各クラスのラベルを持つサンプルを1つずつ取得する。
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

class_ids, sample_indices = np.unique(train_dataset.targets, return_index=True)

fig = plt.figure(figsize=(10, 4))
fig.suptitle(
    "Examples of every class in the Fashion-MNIST dataset", fontsize="x-large"
)

for i in class_ids:
    img = train_dataset.data[sample_indices[i]]
    class_name = train_dataset.classes[i]

    ax = fig.add_subplot(2, 5, i + 1)
    ax.set_title(f"{i}: {class_name}")
    ax.set_axis_off()
    ax.imshow(img, cmap="gray")

plt.show()

In [30]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 10),
            nn.LogSoftmax(dim=1),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)

        return x

In [31]:
device = torch.device("mps")   # if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 10

In [ ]:
train_loss_list = []
train_acc_list = []
for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0

    model.train()
    for images, labels in train_data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels).sum().item()

    train_loss /= len(train_data_loader)
    train_acc /= len(train_data_loader)
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)

    print(f"Epoch: {epoch + 1}/{epochs}")
    print(f"Train loss: {train_loss:.4f}, accuracy: {train_acc:.4f}")

In [ ]:
plt.plot(train_loss_list, label = 'train loss')
plt.plot(train_acc_list, label = 'train acc')
plt.legend()
plt.show()

In [10]:
# 学習データを読み込む DataLoader を作成する。
train_dataset = datasets.FashionMNIST(
    root, train=True, transform=data_transform, download=True
)